In [1]:
import findspark
findspark.init("C:/spark") 

from pyspark import SparkContext
sc=SparkContext("local","Pyspark new")

In [2]:
from pyspark.sql import SparkSession
spark= SparkSession.builder.getOrCreate()

In [3]:
from pyspark.sql.functions import *

In [4]:
nums= [1,2,3,4,5,6,7,8,9,10]
rdd1= sc.parallelize(nums)
print(rdd1.collect())
rdd2= rdd1.map(lambda x: x*x)
rdd2.collect()

[1, 2, 3, 4, 5, 6, 7, 8, 9, 10]


[1, 4, 9, 16, 25, 36, 49, 64, 81, 100]

In [5]:
print(rdd2.max())
print(rdd1.mean())

100
5.5


In [6]:
vals= [('vik',20),('sami',20),('valour',21),('schinder',22),('largo',30)]
cols=['Name','Reg']

df1= spark.createDataFrame(vals,cols)
df1.show()

+--------+---+
|    Name|Reg|
+--------+---+
|     vik| 20|
|    sami| 20|
|  valour| 21|
|schinder| 22|
|   largo| 30|
+--------+---+



In [7]:
print(df1.describe('Reg').show())
df1.printSchema()

+-------+-----------------+
|summary|              Reg|
+-------+-----------------+
|  count|                5|
|   mean|             22.6|
| stddev|4.219004621945797|
|    min|               20|
|    max|               30|
+-------+-----------------+

None
root
 |-- Name: string (nullable = true)
 |-- Reg: long (nullable = true)



In [4]:
from pyspark.sql.types import *

In [9]:
d=spark.read.option('header',True).option('InferSchema',True).csv('customer.csv')
d.printSchema()

root
 |-- Index: integer (nullable = true)
 |-- Customer Id: string (nullable = true)
 |-- First Name: string (nullable = true)
 |-- Last Name: string (nullable = true)
 |-- Company: string (nullable = true)
 |-- City: string (nullable = true)
 |-- Country: string (nullable = true)
 |-- Phone 1: string (nullable = true)
 |-- Phone 2: string (nullable = true)
 |-- Email: string (nullable = true)
 |-- Subscription Date: date (nullable = true)
 |-- Website: string (nullable = true)
 |-- Age: integer (nullable = true)
 |-- Salary: integer (nullable = true)



In [10]:
sch= StructType(
    [
        StructField("Index",IntegerType(),True),
        StructField("Customer Id",StringType(),True),
        StructField("First Name",StringType(),True),   
        StructField("Last Name",StringType(),True),
        StructField("Company",StringType(),True),
        StructField("City",StringType(),True),
        StructField("Country",StringType(),True),
        StructField("Phone 1",StringType(),True),
        StructField("Phone 2",StringType(),True),
        StructField("Email",StringType(),True),
        StructField("Subscription Date",DateType(),True),
        StructField("Website",StringType(),True),
        StructField("Age",IntegerType(),True),
        StructField("Salary",IntegerType(),True)
    ]
)

d= spark.read.option("header",True).schema(sch).csv('customer.csv')

In [11]:
d.show(3)
d.printSchema()

+-----+---------------+----------+---------+---------------+-----------------+-------------------+---------------+-------------------+--------------------+-----------------+--------------------+---+------+
|Index|    Customer Id|First Name|Last Name|        Company|             City|            Country|        Phone 1|            Phone 2|               Email|Subscription Date|             Website|Age|Salary|
+-----+---------------+----------+---------+---------------+-----------------+-------------------+---------------+-------------------+--------------------+-----------------+--------------------+---+------+
|    1|DD37Cf93aecA6Dc|    Sheryl|   Baxter|Rasmussen Group|     East Leonard|              Chile|   229.077.5154|   397.884.0519x718|zunigavanessa@smi...|       2020-08-24|http://www.stephe...| 20| 30000|
|    2|1Ef7b82A4CAAD10|   Preston|   Lozano|    Vega-Gentry|East Jimmychester|           Djibouti|     5153435776|   686-620-1820x944|     vmata@colon.com|       2021-04-23|htt

In [12]:
d.select('First Name','Last Name','Age','Salary').show()

+----------+---------+---+------+
|First Name|Last Name|Age|Salary|
+----------+---------+---+------+
|    Sheryl|   Baxter| 20| 30000|
|   Preston|   Lozano| 20| 35000|
|       Roy|    Berry| 12| 35001|
|     Linda|    Olsen| 13| 35002|
|    Joanna|   Bender| 14| 35003|
|     Aimee|    Downs| 15| 35004|
|    Darren|     Peck| 22| 35005|
|     Brett|   Mullen| 17| 35006|
|    Sheryl|   Meyers| 18| 35007|
|  Michelle|Gallagher| 19| 35008|
|      Carl|Schroeder| 20| 35009|
|     Jenna|   Dodson| 21| 35010|
|    Tracey|     Mata| 22| 35011|
|  Kristine|      Cox| 23| 35012|
|     Faith|     Lutz| 24| 35013|
|   Miranda|  Beasley| 25| 35014|
|  Caroline|    Foley| 26| 35015|
|      Greg|     Mata| 27| 35016|
|  Clifford| Jacobson| 18| 39017|
|    Joanna|     Kirk| 18| 35018|
+----------+---------+---+------+
only showing top 20 rows



In [13]:
d_col= d.withColumn("Age Group", when((col("Age") <= 28) & (col('Age')>17),\
                     "Young Adults").otherwise("Not Young Adults"))

d_col.show(4)

+-----+---------------+----------+---------+--------------------+-----------------+-------------------+--------------------+-------------------+--------------------+-----------------+--------------------+---+------+----------------+
|Index|    Customer Id|First Name|Last Name|             Company|             City|            Country|             Phone 1|            Phone 2|               Email|Subscription Date|             Website|Age|Salary|       Age Group|
+-----+---------------+----------+---------+--------------------+-----------------+-------------------+--------------------+-------------------+--------------------+-----------------+--------------------+---+------+----------------+
|    1|DD37Cf93aecA6Dc|    Sheryl|   Baxter|     Rasmussen Group|     East Leonard|              Chile|        229.077.5154|   397.884.0519x718|zunigavanessa@smi...|       2020-08-24|http://www.stephe...| 20| 30000|    Young Adults|
|    2|1Ef7b82A4CAAD10|   Preston|   Lozano|         Vega-Gentry|Eas

In [14]:
d_filt=d_col.filter(col("Age")>35)
d_filt.select("Customer Id","First Name","Age").show(4)

+---------------+----------+---+
|    Customer Id|First Name|Age|
+---------------+----------+---+
|aBdfcF2c50b0bfD|    Dakota| 36|
|b92EBfdF8a3f0E6| Frederick| 37|
|3B5dAAFA41AFa22|  Stefanie| 38|
|EDA69ca7a6e96a2|      Kent| 39|
+---------------+----------+---+
only showing top 4 rows



In [15]:
print(f"No. of adults abover 35 years: {d_filt.count()}")
d.agg(max('Age')).show()
d.agg(min('Salary')).show()
d.agg(avg('Age')).show()

No. of adults abover 35 years: 58
+--------+
|max(Age)|
+--------+
|     107|
+--------+

+-----------+
|min(Salary)|
+-----------+
|       1000|
+-----------+

+--------+
|avg(Age)|
+--------+
|    49.0|
+--------+



In [ ]:
s=d_col.groupBy('Age Group').agg(
    count("*").alias('Count'),
    max('Salary').alias('Maximum Salary'),
    avg('Age').alias("Average age"),
    min('Age').alias('Minimum Age'),
    min("Salary").alias('Minimun salary')
)
s.show()

+----------------+-----+--------------+------------------+-----------+--------------+
|       Age Group|Count|Maximum Salary|       Average age|Minimum Age|Minimun salary|
+----------------+-----+--------------+------------------+-----------+--------------+
|Not Young Adults|   76|        435066| 57.78947368421053|         10|          1000|
|    Young Adults|   24|        350035|21.166666666666668|         18|         30000|
+----------------+-----+--------------+------------------+-----------+--------------+



In [22]:
rdd3=sc.parallelize([('A',2),('B',22),('C',4),('D',44)])
s=rdd3.flatMap(lambda x:(x[1],x[0]) ).collect() #reversed key vlue and removed nesting
r=rdd3.flatMap(lambda x:x)
print(r.collect())
s

['A', 2, 'B', 22, 'C', 4, 'D', 44]


[2, 'A', 22, 'B', 4, 'C', 44, 'D']

In [23]:
# text file 

rdd_txt= sc.textFile("sample.txt")
rdd_txt.collect()

['TXT test file',
 'Purpose: Provide example of this file type',
 'Document file type: TXT',
 'Version: 1.0',
 'Remark:',
 '',
 'Example content:',
 'The names "John Doe" for males, "Jane Doe" or "Jane Roe" for females, or "Jonnie Doe" and "Janie Doe" for children, or just "Doe" non-gender-specifically are used as placeholder names for a party whose true identity is unknown or must be withheld in a legal action, case, or discussion. The names are also used to refer to acorpse or hospital patient whose identity is unknown. This practice is widely used in the United States and Canada, but is rarely used in other English-speaking countries including the United Kingdom itself, from where the use of "John Doe" in a legal context originates. The names Joe Bloggs or John Smith are used in the UK instead, as well as in Australia and New Zealand.',
 'John Doe is sometimes used to refer to a typical male in other contexts as well, in a similar manner to John Q. Public, known in Great Britain as 

In [27]:
words= rdd_txt.flatMap(lambda x: x.split(" "))
print("No.of words in the file: ",words.count())
a= words.distinct().count()
print(f"No of distinct words: {a}")

#print(words.collect())


No.of words in the file:  430
No of distinct words: 232


In [28]:
aInTxt= words.filter(lambda x: x.lower().startswith('a'))
print("No.of words starting eith 'a' :",aInTxt.count())


No.of words starting eith 'a' : 54


In [29]:
mp= aInTxt.map(lambda x: (x,1))
mp2= mp.reduceByKey(lambda x,y: x+y)
mp2.collect()

[('and', 8),
 ('are', 7),
 ('as', 8),
 ('a', 10),
 ('action,', 2),
 ('also', 3),
 ('acorpse', 1),
 ('Australia', 1),
 ('along', 1),
 ('address', 1),
 ('an', 1),
 ('American', 2),
 ('A', 1),
 ('Additional', 1),
 ('avoid', 1),
 ('anonymous', 2),
 ('alternatives', 1),
 ('Attribution-ShareAlike', 1),
 ('according', 1),
 ('above.', 1)]

### entity resolution 

In [5]:
dff= spark.read.option("InferSchema",True).option("header",True).csv('customer(1).csv')
dff.show()

+---+--------------+-----------+
| id|          name|       city|
+---+--------------+-----------+
|  1|     John Doe |   New York|
|  2|       Jon Doe|   New York|
|  3|   Johnny Doe |Los Angeles|
|  4|   Jane Smith |    Chicago|
|  5|   Janet Smith|    Chicago|
|  6|      John Doe|   New York|
|  7| Jonathon Doe |   New York|
|  8|        Jon Do|   New York|
|  9|       John D.|   New York|
| 10|     John Doe |Los Angeles|
| 11|    Jane Smyth|    Chicago|
| 12|     J. Smith |    Chicago|
| 13|  Janet Smythe|    Chicago|
| 14|  Janie Smith |    Chicago|
| 15|     Johnny Do|Los Angeles|
| 16| Johnathan Doe|     Boston|
| 17|     John Doe |     Boston|
| 18|   John A. Doe|   New York|
| 19| John Doe Jr. |   New York|
| 20|   Johnny Doe |     Newark|
+---+--------------+-----------+
only showing top 20 rows



In [6]:
# cleaning, normalising

dff_cleaned= dff.withColumn("name_cleaned", lower(trim(regexp_replace(col('name'),"[^A-Za-z]"," "))))\
                .withColumn("city_cleaned", lower(trim(regexp_replace(col('city'),"[^A-Za-z0-9]"," "))))

dff_cleaned.show()

+---+--------------+-----------+-------------+------------+
| id|          name|       city| name_cleaned|city_cleaned|
+---+--------------+-----------+-------------+------------+
|  1|     John Doe |   New York|     john doe|    new york|
|  2|       Jon Doe|   New York|      jon doe|    new york|
|  3|   Johnny Doe |Los Angeles|   johnny doe| los angeles|
|  4|   Jane Smith |    Chicago|   jane smith|     chicago|
|  5|   Janet Smith|    Chicago|  janet smith|     chicago|
|  6|      John Doe|   New York|     john doe|    new york|
|  7| Jonathon Doe |   New York| jonathon doe|    new york|
|  8|        Jon Do|   New York|       jon do|    new york|
|  9|       John D.|   New York|       john d|    new york|
| 10|     John Doe |Los Angeles|     john doe| los angeles|
| 11|    Jane Smyth|    Chicago|   jane smyth|     chicago|
| 12|     J. Smith |    Chicago|     j  smith|     chicago|
| 13|  Janet Smythe|    Chicago| janet smythe|     chicago|
| 14|  Janie Smith |    Chicago|  janie 

In [7]:
# blocking
dff_blocked= dff_cleaned.withColumn("block_key",
                        concat_ws(
                            "",
                            trim(regexp_replace(col("city_cleaned"),"\\s+","")).substr(1,4),
                            trim(regexp_replace(col("name_cleaned"),"\\s+","")).substr(1,4)
                        )
            )

dff_blocked.show()


+---+--------------+-----------+-------------+------------+---------+
| id|          name|       city| name_cleaned|city_cleaned|block_key|
+---+--------------+-----------+-------------+------------+---------+
|  1|     John Doe |   New York|     john doe|    new york| newyjohn|
|  2|       Jon Doe|   New York|      jon doe|    new york| newyjond|
|  3|   Johnny Doe |Los Angeles|   johnny doe| los angeles| losajohn|
|  4|   Jane Smith |    Chicago|   jane smith|     chicago| chicjane|
|  5|   Janet Smith|    Chicago|  janet smith|     chicago| chicjane|
|  6|      John Doe|   New York|     john doe|    new york| newyjohn|
|  7| Jonathon Doe |   New York| jonathon doe|    new york| newyjona|
|  8|        Jon Do|   New York|       jon do|    new york| newyjond|
|  9|       John D.|   New York|       john d|    new york| newyjohn|
| 10|     John Doe |Los Angeles|     john doe| los angeles| losajohn|
| 11|    Jane Smyth|    Chicago|   jane smyth|     chicago| chicjane|
| 12|     J. Smith |

In [8]:
c=dff_blocked.select('block_key').distinct()
c.count()

21

In [9]:
# generating candidates
a= dff_blocked.alias('a')
b= dff_blocked.alias('b')
a.show(2); b.show(2)

+---+----------+--------+------------+------------+---------+
| id|      name|    city|name_cleaned|city_cleaned|block_key|
+---+----------+--------+------------+------------+---------+
|  1| John Doe |New York|    john doe|    new york| newyjohn|
|  2|   Jon Doe|New York|     jon doe|    new york| newyjond|
+---+----------+--------+------------+------------+---------+
only showing top 2 rows

+---+----------+--------+------------+------------+---------+
| id|      name|    city|name_cleaned|city_cleaned|block_key|
+---+----------+--------+------------+------------+---------+
|  1| John Doe |New York|    john doe|    new york| newyjohn|
|  2|   Jon Doe|New York|     jon doe|    new york| newyjond|
+---+----------+--------+------------+------------+---------+
only showing top 2 rows



In [10]:
candidates= a.join(b, (col('a.block_key')==col('b.block_key')) & (col('a.id')<col('b.id'))) #.filter(col('a.id')<col('b.id'))
candidates.show()

+---+------------+-----------+------------+------------+---------+---+-------------+-----------+------------+------------+---------+
| id|        name|       city|name_cleaned|city_cleaned|block_key| id|         name|       city|name_cleaned|city_cleaned|block_key|
+---+------------+-----------+------------+------------+---------+---+-------------+-----------+------------+------------+---------+
|  1|   John Doe |   New York|    john doe|    new york| newyjohn| 41|    John Doe!|   New York|    john doe|    new york| newyjohn|
|  1|   John Doe |   New York|    john doe|    new york| newyjohn| 40| John A. D$e |   New York| john a  d e|    new york| newyjohn|
|  1|   John Doe |   New York|    john doe|    new york| newyjohn| 37|     John Dœ |   New York|      john d|    new york| newyjohn|
|  1|   John Doe |   New York|    john doe|    new york| newyjohn| 36|    John* Doe|   New York|   john  doe|    new york| newyjohn|
|  1|   John Doe |   New York|    john doe|    new york| newyjohn| 19

In [11]:
# calculating similarity
candidatePairs= candidates.withColumn('name_Levenshtein', levenshtein(col('a.name_cleaned'),col('b.name_cleaned')))
candidatePairs= candidatePairs.select(
    col('a.id').alias('id_a'),
    col('b.id').alias('id_b'),
    col('a.name_cleaned').alias('name_a'),
    col('b.name_cleaned').alias('name_b'),
    col('a.city_cleaned').alias('city_a'),
    col('b.city_cleaned').alias('city_b'),
    col('a.block_key'),
    col('name_levenshtein').alias('lev')
)
candidatePairs.show()

+----+----+-----------+------------+-----------+-----------+---------+---+
|id_a|id_b|     name_a|      name_b|     city_a|     city_b|block_key|lev|
+----+----+-----------+------------+-----------+-----------+---------+---+
|   1|  41|   john doe|    john doe|   new york|   new york| newyjohn|  0|
|   1|  40|   john doe| john a  d e|   new york|   new york| newyjohn|  4|
|   1|  37|   john doe|      john d|   new york|   new york| newyjohn|  2|
|   1|  36|   john doe|   john  doe|   new york|   new york| newyjohn|  1|
|   1|  19|   john doe| john doe jr|   new york|   new york| newyjohn|  3|
|   1|  18|   john doe| john a  doe|   new york|   new york| newyjohn|  3|
|   1|   9|   john doe|      john d|   new york|   new york| newyjohn|  2|
|   1|   6|   john doe|    john doe|   new york|   new york| newyjohn|  0|
|   2|  32|    jon doe|    jon  doe|   new york|   new york| newyjond|  1|
|   2|   8|    jon doe|      jon do|   new york|   new york| newyjond|  1|
|   3|  33| johnny doe|  

In [13]:
dff_similarity = candidatePairs.withColumn('similarity', 1-(col('lev')/greatest(length(col('name_a')),length(col('name_b')))))

dff_similarity.show()

+----+----+-----------+------------+-----------+-----------+---------+---+------------------+
|id_a|id_b|     name_a|      name_b|     city_a|     city_b|block_key|lev|        similarity|
+----+----+-----------+------------+-----------+-----------+---------+---+------------------+
|   1|  41|   john doe|    john doe|   new york|   new york| newyjohn|  0|               1.0|
|   1|  40|   john doe| john a  d e|   new york|   new york| newyjohn|  4|0.6363636363636364|
|   1|  37|   john doe|      john d|   new york|   new york| newyjohn|  2|              0.75|
|   1|  36|   john doe|   john  doe|   new york|   new york| newyjohn|  1|0.8888888888888888|
|   1|  19|   john doe| john doe jr|   new york|   new york| newyjohn|  3|0.7272727272727273|
|   1|  18|   john doe| john a  doe|   new york|   new york| newyjohn|  3|0.7272727272727273|
|   1|   9|   john doe|      john d|   new york|   new york| newyjohn|  2|              0.75|
|   1|   6|   john doe|    john doe|   new york|   new york|

In [15]:
# classification 
thresh= 0.8
dff_classified= dff_similarity.withColumn("isMatch",when( col('similarity')>thresh, 1).otherwise(0))

dff_classified.show()   

+----+----+-----------+------------+-----------+-----------+---------+---+------------------+-------+
|id_a|id_b|     name_a|      name_b|     city_a|     city_b|block_key|lev|        similarity|isMatch|
+----+----+-----------+------------+-----------+-----------+---------+---+------------------+-------+
|   1|  41|   john doe|    john doe|   new york|   new york| newyjohn|  0|               1.0|      1|
|   1|  40|   john doe| john a  d e|   new york|   new york| newyjohn|  4|0.6363636363636364|      0|
|   1|  37|   john doe|      john d|   new york|   new york| newyjohn|  2|              0.75|      0|
|   1|  36|   john doe|   john  doe|   new york|   new york| newyjohn|  1|0.8888888888888888|      1|
|   1|  19|   john doe| john doe jr|   new york|   new york| newyjohn|  3|0.7272727272727273|      0|
|   1|  18|   john doe| john a  doe|   new york|   new york| newyjohn|  3|0.7272727272727273|      0|
|   1|   9|   john doe|      john d|   new york|   new york| newyjohn|  2|        

In [16]:
linked= dff_classified.filter(col("isMatch")==1).select("id_a","id_b").collect()
#linked

In [19]:
parent= {}

def find(x):
    if parent.get(x,x) != x:
        parent[x] = find(parent[x])
    return parent.get(x,x)

def union(x,y):
    px,py= find(x), find(y)

    if px!=py:
        parent[py]=px

for r in linked:
    union(r['id_a'],r['id_b'])

clusters={}

for node in parent:
    root=find(node)
    clusters.setdefault(root,[]).append(node)

In [44]:
parent = {}
    
def find(x):
    if parent.get(x, x) != x:
        parent[x] = find(parent[x])
    return parent.get(x, x)

def union(x, y):
    px, py = find(x), find(y)
    if px != py:
        parent[py] = px

for row in linked:
    union(row["id_a"], row["id_b"])

clusters = {}
for node in parent:
    root = find(node)
    clusters.setdefault(root, []).append(node)

print(clusters)

{18: [41, 36, 6, 40, 1], 2: [32, 8], 3: [33, 15], 4: [35, 11, 5, 13], 9: [37], 17: [39], 20: [42], 21: [43], 22: [44], 24: [46], 25: [47], 26: [48], 27: [49], 28: [50]}


## ALS

In [4]:
from pyspark.ml.recommendation import *
from pyspark.ml.evaluation import RegressionEvaluator
from pyspark.sql.types import *

In [5]:
df2= spark.read.json('movies.json')
df2.printSchema()

root
 |-- helpfulness: string (nullable = true)
 |-- product_id: string (nullable = true)
 |-- profile_name: string (nullable = true)
 |-- review: string (nullable = true)
 |-- score: double (nullable = true)
 |-- summary: string (nullable = true)
 |-- time: long (nullable = true)
 |-- user_id: string (nullable = true)



In [6]:
df2.select('product_id','user_id','score').show(4)

+----------+--------------+-----+
|product_id|       user_id|score|
+----------+--------------+-----+
|B003AI2VGA|A141HP4LYPWMSR|  3.0|
|B003AI2VGA|A328S9RN3U5M68|  3.0|
|B003AI2VGA|A1I7QGUDP043DG|  5.0|
|B003AI2VGA|A1M5405JH9THP9|  3.0|
+----------+--------------+-----+
only showing top 4 rows



In [7]:
df2_hashed= df2.withColumn("user_hash", sha1(col('user_id'))).withColumn("prod_hash",sha1(col('product_id')))
df2_hashed.select('product_id','prod_hash','user_id','user_hash').show(4)

+----------+--------------------+--------------+--------------------+
|product_id|           prod_hash|       user_id|           user_hash|
+----------+--------------------+--------------+--------------------+
|B003AI2VGA|3a1339ccd83d1fb05...|A141HP4LYPWMSR|dcb0cc72376dc54f3...|
|B003AI2VGA|3a1339ccd83d1fb05...|A328S9RN3U5M68|a20670dd0e673bd6a...|
|B003AI2VGA|3a1339ccd83d1fb05...|A1I7QGUDP043DG|616addd183663c420...|
|B003AI2VGA|3a1339ccd83d1fb05...|A1M5405JH9THP9|5318ec702ebd83352...|
+----------+--------------------+--------------+--------------------+
only showing top 4 rows



In [8]:
hex_to_bigInt= udf(lambda v: int(v,16)%(10**8), LongType())

In [9]:
df2_clean= df2_hashed.withColumn('userhash_int', hex_to_bigInt(col('user_hash')))
df2_clean= df2_clean.withColumn('prodhash_int', hex_to_bigInt(col('prod_hash')))

df2_clean.select('product_id','prodhash_int','user_id','userhash_int').show(4)

+----------+------------+--------------+------------+
|product_id|prodhash_int|       user_id|userhash_int|
+----------+------------+--------------+------------+
|B003AI2VGA|    51259877|A141HP4LYPWMSR|     5460385|
|B003AI2VGA|    51259877|A328S9RN3U5M68|    64843361|
|B003AI2VGA|    51259877|A1I7QGUDP043DG|     1480848|
|B003AI2VGA|    51259877|A1M5405JH9THP9|    81925650|
+----------+------------+--------------+------------+
only showing top 4 rows



In [10]:
# moving on to als

(train, test)= df2_clean.randomSplit([0.8,0.2])

In [11]:
als_model= ALS(
    userCol='userhash_int', itemCol='prodhash_int',
    ratingCol='score',
    maxIter=10, rank=10, regParam=0.1,
    coldStartStrategy="drop"
)

In [12]:
mod=als_model.fit(train)

In [13]:
prd= mod.transform(test)


In [14]:
prd.select('product_id','user_id','score','prediction').show(7)

+----------+--------------+-----+----------+
|product_id|       user_id|score|prediction|
+----------+--------------+-----+----------+
|B004R10MA6|A2N6K4RFSPY2GV|  5.0| 2.6359391|
|B000VULA9E|A2B73CL3QSYWLB|  2.0| 3.9864812|
|B0000DK4QI|A3EJYJC25OJVKK|  5.0|  5.333603|
|B00004CZR1|A2EY23WUHTEHAY|  5.0|  4.893707|
|B00009OWJT|A2B73CL3QSYWLB|  1.0| -2.246769|
|B0041KKZH8|A2N6K4RFSPY2GV|  4.0|    4.5465|
|B000CEXFBU|A2B73CL3QSYWLB|  4.0| 3.0163393|
+----------+--------------+-----+----------+
only showing top 7 rows



In [15]:
evl= RegressionEvaluator(
    metricName='rmse',
    labelCol='score',
    predictionCol='prediction'
)

In [16]:
rmse=evl.evaluate(prd)

In [17]:
rmse

1.9349880881874353

In [19]:
# hyper parameter tuning using crossvalidation and grid search
from pyspark.ml.tuning import ParamGridBuilder, CrossValidator

In [ ]:
als_tuning= ALS(
    userCol='userhash_int', itemCol='prodhash_int',ratingCol='score',
    maxIter=10,rank=10,regParam=0.1,coldStartStrategy='drop'
)

In [21]:
grid= ParamGridBuilder().addGrid(als_tuning.rank,[3,8]).addGrid(als_tuning.maxIter,[10,20]).\
        addGrid(als_tuning.regParam, [0.1,1]).build()

In [22]:
evl_tuned= RegressionEvaluator(
    metricName= 'rmse', labelCol='score',predictionCol='prediction'
)

In [23]:
cross= CrossValidator(
    estimator=als_tuning, estimatorParamMaps=grid, numFolds=3, evaluator=evl_tuned
)

In [24]:
models=cross.fit(train)

In [25]:
preds= models.transform(test)

In [26]:
rmse_tuning= evl_tuned.evaluate(preds)
print(rmse_tuning)

1.573530909141415


In [4]:
logs = [
"john,2025-07-01,login|search|click",
"mary,2025-07-01,search|logout",
"john,2025-07-02,login|click|click",
"jacob,2025-07-03,login|click|click"
]


In [5]:
rdd = sc.parallelize(logs)
rdd = rdd.map(lambda x: x.split(",")) 
rdd = rdd.map(lambda x: (x[0], x[1], x[2].split("|")))

In [6]:
rdd.collect()

[('john', '2025-07-01', ['login', 'search', 'click']),
 ('mary', '2025-07-01', ['search', 'logout']),
 ('john', '2025-07-02', ['login', 'click', 'click']),
 ('jacob', '2025-07-03', ['login', 'click', 'click'])]